In [1]:
import os

In [2]:
%pwd

'/Users/bhikipallai/Desktop/Projects/NLP_projetcs/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/Users/bhikipallai/Desktop/Projects/NLP_projetcs'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml_file, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml_file(config_filepath)
        self.params = read_yaml_file(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.evaluation_strategy,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )

        return model_trainer_config

In [8]:
!pip uninstall -y transformers accelerate

Found existing installation: transformers 4.44.1
Uninstalling transformers-4.44.1:
  Successfully uninstalled transformers-4.44.1
Found existing installation: accelerate 0.33.0
Uninstalling accelerate-0.33.0:
  Successfully uninstalled accelerate-0.33.0


In [9]:
%pip install transformers accelerate

  Using cached transformers-4.44.1-py3-none-any.whl.metadata (43 kB)
  Using cached accelerate-0.33.0-py3-none-any.whl.metadata (18 kB)
Using cached transformers-4.44.1-py3-none-any.whl (9.5 MB)
Using cached accelerate-0.33.0-py3-none-any.whl (315 kB)
Note: you may need to restart the kernel to use updated packages.


In [10]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

/Applications/anaconda3/envs/nlp/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-08-22 08:18:48,266: INFO: config: PyTorch version 2.4.0 available.]


In [11]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        # Use 'mps' explicitly if running on Apple Silicon
        device = "mps" if torch.backends.mps.is_available() else "cpu"
        
        # Initialize tokenizer and model
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)
        
        # Load dataset
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        # Define training arguments with memory optimizations
        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir,
            num_train_epochs=float(self.config.num_train_epochs),
            warmup_steps=int(self.config.warmup_steps),
            per_device_train_batch_size=1,  # Reduce batch size
            per_device_eval_batch_size=1,   # Reduce batch size
            weight_decay=float(self.config.weight_decay),
            logging_steps=int(self.config.logging_steps),
            evaluation_strategy="steps",
            eval_steps=500,
            save_steps=int(1e6),  # Adjust to save less frequently
            gradient_accumulation_steps=1,  # Adjust if needed
            fp16=False  # Mixed precision not supported on MPS
        )

        # Initialize Trainer
        trainer = Trainer(
            model=model_pegasus,
            args=trainer_args,
            tokenizer=tokenizer,
            data_collator=seq2seq_data_collator,
            train_dataset=dataset_samsum_pt["test"], 
            eval_dataset=dataset_samsum_pt["validation"]
        )
        
        # Start training
        trainer.train()

        # Save model and tokenizer
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir, "pegasus-samsum-model"))
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))


In [12]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-08-22 08:18:48,476: INFO: common: yaml file: config/config.yaml loaded successfully]


[2024-08-22 08:18:48,488: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-22 08:18:48,490: INFO: common: created directory at: artifacts]
[2024-08-22 08:18:48,491: INFO: common: created directory at: artifacts/model_trainer]


/Applications/anaconda3/envs/nlp/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/Applications/anaconda3/envs/nlp/lib/python3.12/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
100%|██████████| 9/9 [01:40<00:00, 11.17s/it]


{'train_runtime': 100.5237, 'train_samples_per_second': 0.081, 'train_steps_per_second': 0.09, 'train_loss': 7.770673116048177, 'epoch': 0.01}


In [13]:
import transformers
import accelerate

print(transformers.__version__)
print(accelerate.__version__)


4.44.1
0.33.0
